In [1]:
from typing import List
from copy import deepcopy
import pandas as pd

import ase
from ase import Atoms
from ase.calculators.lj import LennardJones
from ase.io import Trajectory
from ase.calculators.calculator import Calculator, all_changes

import numpy as np
import matplotlib.pyplot as plt

from cascade.utils import canonicalize

%load_ext autoreload
%autoreload 2

In [2]:
atoms = Atoms('N2')

In [3]:
atoms.positions

array([[0., 0., 0.],
       [0., 0., 0.]])

In [4]:
def sample_diatomic_distances(
    atoms_str: str = 'N2', 
    dist_range: tuple = (1.1, 3), 
    n: int = 100) -> List[ase.Atoms]:     
    """Create a trajectory of a diatomic system with linearly sampled distances
    atoms_str: ase specification of diatomic
    dist_range: (angsroms) range of distances to sample
    n: number of samples
    """
    atoms = Atoms(atoms_str)
    trajectory = []
    x_vals = np.linspace(*dist_range, n)
    for x in x_vals: 
        atoms = atoms.copy()
        atoms.set_positions([[0,0,0], [x, 0, 0]])
        trajectory.append(atoms)
    return trajectory

In [5]:
def apply_calculator(
    calc: ase.calculators.calculator.Calculator, 
    traj: List[ase.Atoms]) -> List[Atoms]:
    """Run a calculator on every atoms object in a list"""

    traj = deepcopy(traj)
    out = []
    for atoms in traj: 
        atoms.calc = calc
        atoms.get_forces()
        atoms = canonicalize(atoms)
        out.append(atoms)
    return out

In [6]:
lj1, lj2  = LennardJones(sigma=1, eps=1), LennardJones(sigma=1, epsilon=1.1)

In [7]:
traj_base = sample_diatomic_distances()
traj_lj1  = apply_calculator(lj1, traj_base)
traj_lj2  = apply_calculator(lj2, traj_base)

In [8]:
traj_lj1[:3]

[Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False)]

In [9]:
atoms.calc is None

True

In [ ]:
plt.plot(traj_lj1['position'], traj_lj1['force'], label='$\epsilon=1$')
plt.plot(traj_lj2['position'], traj_lj2['force'], label='$\epsilon=1.1$')
plt.xlabel('$R_\mathrm{{N-N}}$ ($\mathrm{\AA}$)')
plt.ylabel('$F_{LJ}$ (eV/$\mathrm{\AA}$)')
plt.legend()
plt.title('Lennard Jones Forces acting on N-N for varying $\epsilon$');

In [ ]:
plt.plot(traj_lj1['position'], (traj_lj1['force'] - traj_lj2['force']).abs(), label='$\epsilon=1$')
plt.title('Absolute force difference given $R_\mathrm{{N-N}}$ across LJ potentials')
plt.ylabel('$|F^{\epsilon=1}_{LJ} - F^{\epsilon=1.1}_{LJ}|$ (eV/$\mathrm{\AA}$)')
plt.xlabel('$R_\mathrm{{N-N}}$ ($\mathrm{\AA}$)')

## Ensemble Class

In [ ]:
from ase.calculators import EnsembleCalculator

In [ ]:
ens = EnsembleCalculator([lj1, lj2])

In [ ]:
traj_ens = apply_calculator(ens, traj_base)

In [ ]:
traj_ens

In [ ]:
traj_ens['UQ'] = traj_ens['atoms'].map(lambda a: a.info['forces_std'][0,0])

In [ ]:
plt.plot(traj_lj1['position'], (traj_lj1['force'] - traj_lj2['force']).abs(), label='$|F^{\epsilon=1}_{LJ} - F^{\epsilon=1.1}_{LJ}|$')
plt.plot(traj_ens['position'], traj_ens['UQ'], label='std($F_\mathrm{ens}$)[0,0]')
plt.ylabel('$F eV/$\mathrm{\AA}$)')
plt.xlabel('$R_\mathrm{{N-N}}$ ($\mathrm{\AA}$)')
plt.legend()

In [ ]:
traj_ens['atoms'][0].calc

## TODO import the auditors and try them out